**TCN**

In [ ]:
# =========================
# LG Aimers — TCN Forecast (MIMO & Recursive, 규정 준수)
# =========================

import os, glob, math, copy, random
from typing import Tuple

import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import holidays

# =========================
# 경로/하이퍼파라미터 설정
# =========================
DATA_DIR = '.'
TRAIN_CSV = os.path.join(DATA_DIR, "train", "train.csv")
TEST_GLOB = os.path.join(DATA_DIR, "test", "TEST_*.csv")
SAMPLE_SUB = os.path.join(DATA_DIR, "Submission", "sample_submission.csv")
OUT_SUB = "submission_tcn.csv"

TIME_COL = "영업일자"
ID_COL = "영업장명_메뉴명"
TARGET_COL = "매출수량"

LOOKBACK = 28
HORIZON  = 7

EPOCHS      = 50
BATCH_SIZE  = 1024
LR          = 1e-3
DROPOUT     = 0.2
HIDDEN_CH   = 128
NUM_LEVELS  = 4
KERNEL_SIZE = 3
EMB_DIM     = 32
WEIGHT_DECAY = 1e-4
VAL_RATIO    = 0.10  # 아이템별 뒤쪽 10%를 검증으로 홀드아웃
SMAPE_IGNORE_ZERO_TARGETS = True   # ← 여기만 True/False로 토글
SMAPE_TAU = 1e-6

# === H=1 전용 보조손실 설정 ===
UNROLL_STEPS = 7
SS_RATIO = 0.5          # teacher forcing 비율
AUX_LAMBDA  = 0.3       # 보조손실 가중치

SEED   = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

# 예측 모드: 'mimo' | 'recursive' | 'blend' (blend는 평균 앙상블)
PREDICT_MODE = "blend"   # 필요에 따라 바꿔서 비교

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DEVICE: cuda


In [3]:
# =========================
# 유틸
# =========================

# 시드
def set_seed(seed: int = SEED):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# 파생변수
def make_long_holiday(date_s: pd.Series) -> pd.Series:
    """연휴여부: 공휴일/주말 ∪ 샌드위치(앞뒤가 쉬는 날)"""
    s = pd.to_datetime(date_s)
    start = s.min() - pd.Timedelta(days=7)
    end   = s.max() + pd.Timedelta(days=7)
    cal_dates = pd.date_range(start, end, freq='D')

    kr = holidays.KR(years=cal_dates.year.unique())
    is_holiday = pd.Series([d.date() in kr for d in cal_dates], index=cal_dates)
    is_weekend = pd.Series(cal_dates.weekday >= 5, index=cal_dates)
    rest = is_holiday | is_weekend

    long_flag = (rest | (rest.shift(1, fill_value=False) & rest.shift(-1, fill_value=False))).astype(int)

    key = long_flag.copy(); key.index = key.index.normalize()
    out = s.dt.normalize().map(key).fillna(0).astype(int)
    out.index = date_s.index
    out.name = '연휴여부'
    return out

def add_calendar_features(df: pd.DataFrame) -> pd.DataFrame:
    d = pd.to_datetime(df[TIME_COL])
    dow = d.dt.weekday  # 0=Mon
    df["dow_sin"] = np.sin(2 * np.pi * dow / 7)
    df["dow_cos"] = np.cos(2 * np.pi * dow / 7)
    mon = d.dt.month
    df["mon_sin"] = np.sin(2 * np.pi * mon / 12)
    df["mon_cos"] = np.cos(2 * np.pi * mon / 12)
    df["is_weekend"] = (dow >= 5).astype(np.float32)
    df["holidays"] = make_long_holiday(df[TIME_COL]).astype(np.float32)
    return df

# 손실함수 (타깃0제외)
def smape_loss(pred: torch.Tensor, target: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    with torch.no_grad():
        if SMAPE_IGNORE_ZERO_TARGETS:
            mask = (target.abs() > SMAPE_TAU)  # 타깃≈0 제외
        else:
            mask = ~((pred.abs() <= SMAPE_TAU) & (target.abs() <= SMAPE_TAU))  # 0–0만 제외
    if mask.sum().item() == 0:
        return torch.zeros((), dtype=pred.dtype, device=pred.device)

    p = pred[mask]; t = target[mask]
    denom = (p.abs() + t.abs()).clamp_min(eps)
    diff  = (p - t).abs()
    return (2.0 * diff / denom).mean()

def smape_np(pred, true, eps: float = 1e-6) -> float:
    pred = np.asarray(pred); true = np.asarray(true)
    if SMAPE_IGNORE_ZERO_TARGETS:
        mask = (np.abs(true) > SMAPE_TAU)
    else:
        mask = ~((np.abs(pred) <= SMAPE_TAU) & (np.abs(true) <= SMAPE_TAU))  # 0–0만 제외
    if mask.sum() == 0:
        return np.nan
    p, t = pred[mask], true[mask]
    return float(np.mean(2*np.abs(p-t)/(np.abs(p)+np.abs(t)+eps)))

# 슬라이드 윈도우 생성
def make_windows_per_id(num_mat: np.ndarray, feat_mat: np.ndarray,
                        lookback: int, horizon: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    N = num_mat.shape[0]
    Xn, Xf, Y = [], [], []
    end = N - lookback - horizon + 1
    for i in range(end):
        Xn.append(num_mat[i:i+lookback, :])
        Xf.append(feat_mat[i:i+lookback, :])
        Y.append(num_mat[i+lookback:i+lookback+horizon, 0])
    if end <= 0:
        return (np.zeros((0, lookback, 1), np.float32),
                np.zeros((0, lookback, feat_mat.shape[1]), np.float32),
                np.zeros((0, horizon), np.float32))
    return (np.array(Xn, np.float32), np.array(Xf, np.float32), np.array(Y, np.float32))


def _norm_id(s: str) -> str:
    return str(s).replace('\u00a0',' ').replace('\t',' ').strip()

def normalize_id_columns(df: pd.DataFrame, id_col: str) -> pd.DataFrame:
    if id_col in df.columns:
        df[id_col] = df[id_col].map(_norm_id)
    return df


def approx_shop_weight(item_name: str):
    shop = item_name.split("_", 1)[0] if "_" in item_name else item_name
    return 2.0 if shop in ("담하","미라시아") else 1.0

def make_future_calendar_rows(last_date: pd.Timestamp, days: int = 7):
    idx = pd.date_range(last_date + pd.Timedelta(days=1), periods=days, freq="D")
    df = pd.DataFrame({TIME_COL: idx})
    d = df[TIME_COL]; dow = d.dt.weekday; mon = d.dt.month
    df["dow_sin"] = np.sin(2*np.pi*dow/7);  df["dow_cos"] = np.cos(2*np.pi*dow/7)
    df["mon_sin"] = np.sin(2*np.pi*mon/12); df["mon_cos"] = np.cos(2*np.pi*mon/12)
    df["is_weekend"] = (dow >= 5).astype(np.float32)
    df["holidays"]  = make_long_holiday(df[TIME_COL]).astype(np.float32)
    return df

@torch.no_grad()
def recursive_predict_7days(model_h1: nn.Module,
                            last_28_sales: np.ndarray,  # (28,1)
                            last_28_feats: np.ndarray,  # (28,F)
                            item_idx: int,
                            last_date: pd.Timestamp,
                            feat_cols=None) -> np.ndarray:
    feat_cols = FEAT_COLS if feat_cols is None else feat_cols
    model_h1.eval()
    preds = []
    cur_sales = last_28_sales.copy()
    cur_feats = last_28_feats.copy()
    future_feats = make_future_calendar_rows(last_date, 7)[feat_cols].values.astype(np.float32)
    for step in range(7):
        Xn = torch.from_numpy(cur_sales[None, ...]).to(DEVICE)
        Xf = torch.from_numpy(cur_feats[None, ...]).to(DEVICE)
        ii = torch.tensor([item_idx], dtype=torch.long, device=DEVICE)
        y1 = model_h1(Xn, Xf, ii).detach().cpu().numpy()[0, 0]
        preds.append(float(y1))
        cur_sales = np.vstack([cur_sales[1:], np.array([[y1]], dtype=np.float32)])
        cur_feats = np.vstack([cur_feats[1:], future_feats[step:step+1]])
    out = np.array(preds, dtype=np.float32)
    return np.nan_to_num(out, copy=False).clip(min=0.0)

In [4]:
# =========================
# 모델(TCN)
# =========================
class Chomp1d(nn.Module):
    def __init__(self, chomp_size: int):
        super().__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous() if self.chomp_size > 0 else x

class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, dropout):
        super().__init__()
        pad = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size, padding=pad, dilation=dilation)
        self.chomp1 = Chomp1d(pad); self.relu1 = nn.ReLU(); self.drop1 = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size, padding=pad, dilation=dilation)
        self.chomp2 = Chomp1d(pad); self.relu2 = nn.ReLU(); self.drop2 = nn.Dropout(dropout)
        self.down = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else None
        self.out_relu = nn.ReLU()

    def forward(self, x):
        y = self.conv1(x); y = self.chomp1(y); y = self.relu1(y); y = self.drop1(y)
        y = self.conv2(y); y = self.chomp2(y); y = self.relu2(y); y = self.drop2(y)
        res = x if self.down is None else self.down(x)
        return self.out_relu(y + res)

class GlobalTCN(nn.Module):
    def __init__(self, num_items: int, horizon: int, in_feat: int,
                 emb_dim: int = EMB_DIM, num_levels: int = NUM_LEVELS,
                 hidden_ch: int = HIDDEN_CH, kernel_size: int = KERNEL_SIZE,
                 dropout: float = DROPOUT):
        super().__init__()
        self.item_emb = nn.Embedding(num_items, emb_dim)
        tcn_in = in_feat + emb_dim
        layers = []
        in_ch = tcn_in
        for i in range(num_levels):
            layers.append(TemporalBlock(in_ch, hidden_ch, kernel_size, dilation=2**i, dropout=dropout))
            in_ch = hidden_ch
        self.tcn = nn.Sequential(*layers)
        self.head = nn.Sequential(
            nn.Linear(hidden_ch, horizon),
            nn.Softplus()
        )
    def forward(self, x_num, x_feat, item_idx):
        B, T, _ = x_num.shape
        emb = self.item_emb(item_idx)
        emb_rep = emb.unsqueeze(1).expand(B, T, emb.size(-1))
        x = torch.cat([x_num, x_feat, emb_rep], dim=2)
        x = x.transpose(1, 2)
        h = self.tcn(x)
        h = h[:, :, -1]
        out = self.head(h)
        return out

In [5]:
# =========================
# Dataset / Dataloader
# =========================
class GlobalTCNDataset(Dataset):
    def __init__(self, X_num, X_feat, y, id_idx_arr):
        self.X_num      = X_num
        self.X_feat     = X_feat
        self.y          = y
        self.id_idx_arr = id_idx_arr
    def __len__(self):
        return self.X_num.shape[0]
    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X_num[idx]),
            torch.from_numpy(self.X_feat[idx]),
            torch.tensor(self.id_idx_arr[idx], dtype=torch.long),
            torch.from_numpy(self.y[idx])
        )

In [6]:
# =========================
# train / eval
# =========================
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best = math.inf
        self.count = 0
        self.state = None
    def step(self, metric: float, model: nn.Module) -> bool:
        if (self.best - metric) > self.min_delta:
            self.best = metric; self.count = 0
            self.state = copy.deepcopy(model.state_dict())
            return False
        self.count += 1
        return self.count > self.patience
    def restore(self, model: nn.Module):
        if self.state is not None:
            model.load_state_dict(self.state)

@torch.no_grad()
def evaluate(model: nn.Module, loader: DataLoader, device: str) -> float:
    model.eval(); losses = []
    for xn, xf, ii, y in loader:
        xn, xf, ii, y = xn.to(device), xf.to(device), ii.to(device), y.to(device)
        pred = model(xn, xf, ii)
        loss = smape_loss(pred, y)
        losses.append(loss.item())
    return float(np.mean(losses)) if losses else 0.0

def fit_model(model, train_loader, val_loader, epochs, lr, device):
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    es  = EarlyStopping(patience=10, min_delta=0.0)
    scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
    for ep in range(1, epochs + 1):
        model.train(); tr_losses = []
        for xn, xf, ii, y in train_loader:
            xn, xf, ii, y = xn.to(device), xf.to(device), ii.to(device), y.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(device == "cuda")):
                pred = model(xn, xf, ii)
                loss = smape_loss(pred, y)
            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(opt); scaler.update()
            tr_losses.append(loss.item())
        sch.step()
        val_loss = evaluate(model, val_loader, device)
        print(f"[{ep:03d}] train={np.mean(tr_losses):.6f} | val={val_loss:.6f}")
        if es.step(val_loss, model):
            print(f"Early stopped @ {ep}, best val={es.best:.6f}")
            break
    es.restore(model)
    return model

In [21]:
# 추가 : H=1(재귀) 전용 ‘7‑step Unroll’ 보조손실
@torch.no_grad()
def evaluate_h1(model, loader, device):
    model.eval(); losses=[]
    for xn, xf, ii, y in loader:       # y:(B,7)
        xn, xf, ii, y = xn.to(device), xf.to(device), ii.to(device), y.to(device)
        pred = model(xn, xf, ii)       # (B,1)
        losses.append(smape_loss(pred, y[:, :1]).item())
    return float(np.mean(losses)) if losses else 0.0

def fit_model_h1(model, train_loader, val_loader, epochs, lr, device):
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    es  = EarlyStopping(patience=10, min_delta=0.0)
    scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))

    for ep in range(1, epochs+1):
        model.train(); tr_losses=[]
        for xn, xf, ii, y in train_loader:     # y:(B,7)
            xn, xf, ii, y = xn.to(device), xf.to(device), ii.to(device), y.to(device)
            opt.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=(device=="cuda")):
                # 1-step 손실
                pred1 = model(xn, xf, ii)
                loss  = smape_loss(pred1, y[:, :1])

                # 보조손실
                step_loss = 0.0
                cur_sales, cur_feats = xn.clone(), xf.clone()
                for s in range(UNROLL_STEPS):
                    p = model(cur_sales, cur_feats, ii)[:,0:1]
                    tgt = y[:, s:s+1]
                    step_loss += smape_loss(p, tgt)
                    next_token = tgt if (random.random() < SS_RATIO) else p.detach()
                    cur_sales = torch.cat([cur_sales[:,1:,:], next_token.unsqueeze(1)], dim=1)
                    cur_feats = torch.cat([cur_feats[:,1:,:], cur_feats[:,-1:,:]], dim=1)
                loss = loss + AUX_LAMBDA * (step_loss/UNROLL_STEPS)

            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(),1.0)
            scaler.step(opt); scaler.update()
            tr_losses.append(loss.item())

        sch.step()
        val_loss = evaluate_h1(model, val_loader, device)   # 전용 평가 함수
        print(f"[H1 {ep:03d}] train={np.mean(tr_losses):.6f} | val={val_loss:.6f}")
        if es.step(val_loss, model):
            print(f"H1 Early stopped @ {ep}, best val={es.best:.6f}")
            break

    es.restore(model)
    return model

In [22]:
# -------------------------
# Data builders (H=7 & H=1)
# -------------------------
def build_time_split_windows(train_df, feat_cols, horizon=7, lookback=28, val_ratio=0.10, id2idx=None):
    Xn_tr_list, Xf_tr_list, Y_tr_list, I_tr_list = [], [], [], []
    Xn_vl_list, Xf_vl_list, Y_vl_list, I_vl_list = [], [], [], []

    for k, g in train_df.groupby(ID_COL, sort=False):
        g = g.sort_values(TIME_COL)
        num_mat  = g[[TARGET_COL]].values.astype(np.float32)
        feat_mat = g[feat_cols].values.astype(np.float32)
        Xn, Xf, Y = make_windows_per_id(num_mat, feat_mat, lookback, horizon)
        if len(Xn) == 0:
            continue
        m = len(Xn)
        split = max(1, int(m * (1 - val_ratio)))
        idx_arr = np.full((m,), id2idx.get(k, 0), dtype=np.int64)
        Xn_tr_list.append(Xn[:split]); Xf_tr_list.append(Xf[:split]); Y_tr_list.append(Y[:split]); I_tr_list.append(idx_arr[:split])
        Xn_vl_list.append(Xn[split:]); Xf_vl_list.append(Xf[split:]); Y_vl_list.append(Y[split:]); I_vl_list.append(idx_arr[split:])

    Fc = len(feat_cols)
    def _v(lst, shape): return np.vstack(lst) if lst else np.zeros(shape, np.float32)

    Xn_tr = _v(Xn_tr_list, (0, lookback, 1))
    Xf_tr = _v(Xf_tr_list, (0, lookback, Fc))
    Y_tr  = _v(Y_tr_list,  (0, horizon))
    I_tr  = np.concatenate(I_tr_list) if I_tr_list else np.zeros((0,), np.int64)

    Xn_vl = _v(Xn_vl_list, (0, lookback, 1))
    Xf_vl = _v(Xf_vl_list, (0, lookback, Fc))
    Y_vl  = _v(Y_vl_list,  (0, horizon))
    I_vl  = np.concatenate(I_vl_list) if I_vl_list else np.zeros((0,), np.int64)

    return (Xn_tr, Xf_tr, Y_tr, I_tr), (Xn_vl, Xf_vl, Y_vl, I_vl)

def train_h7_model(train_df, feat_cols, lookback=28, val_ratio=0.10):
    (Xn_tr, Xf_tr, Y_tr, I_tr), (Xn_vl, Xf_vl, Y_vl, I_vl) = build_time_split_windows(
        train_df, feat_cols, horizon=7, lookback=lookback, val_ratio=val_ratio, id2idx=id2idx
    )
    print("H=7 Train windows:", Xn_tr.shape, Xf_tr.shape, Y_tr.shape)
    print("H=7 Valid windows:", Xn_vl.shape, Xf_vl.shape, Y_vl.shape)

    ds_tr = GlobalTCNDataset(Xn_tr, Xf_tr, Y_tr, I_tr)
    ds_vl = GlobalTCNDataset(Xn_vl, Xf_vl, Y_vl, I_vl)
    dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, pin_memory=(DEVICE=="cuda"))
    dl_vl = DataLoader(ds_vl, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=(DEVICE=="cuda"))

    in_feat = 1 + len(feat_cols)
    model_h7 = GlobalTCN(num_items=num_items, horizon=7, in_feat=in_feat,
                         emb_dim=EMB_DIM, num_levels=NUM_LEVELS, hidden_ch=HIDDEN_CH,
                         kernel_size=KERNEL_SIZE, dropout=DROPOUT).to(DEVICE)
    model_h7 = fit_model(model_h7, dl_tr, dl_vl, epochs=EPOCHS, lr=LR, device=DEVICE)
    return model_h7

# 추가 : H=1(재귀) 전용 ‘7‑step Unroll’ 보조손실 추가
def build_windows_for_h1(train_df, feat_cols, lookback=28, val_ratio=0.10, id2idx=None):
    # 기존 horizon=1 → 7 로 변경
    return build_time_split_windows(
        train_df, feat_cols, horizon=7, lookback=lookback, val_ratio=val_ratio, id2idx=id2idx)


def train_h1_model(train_df, feat_cols, lookback=28, val_ratio=0.10):
    (Xn_tr, Xf_tr, Y_tr, I_tr), (Xn_vl, Xf_vl, Y_vl, I_vl) = build_windows_for_h1(
        train_df, feat_cols, lookback=lookback, val_ratio=val_ratio, id2idx=id2idx
    )
    print("H=1(uses y7) Train windows:", Xn_tr.shape, Xf_tr.shape, Y_tr.shape)
    print("H=1(uses y7) Valid windows:", Xn_vl.shape, Xf_vl.shape, Y_vl.shape)

    ds_tr = GlobalTCNDataset(Xn_tr, Xf_tr, Y_tr, I_tr)  # Y_*는 (·,7)
    ds_vl = GlobalTCNDataset(Xn_vl, Xf_vl, Y_vl, I_vl)
    dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, pin_memory=(DEVICE=="cuda"))
    dl_vl = DataLoader(ds_vl, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=(DEVICE=="cuda"))

    in_feat = 1 + len(feat_cols)
    model_h1 = GlobalTCN(num_items=num_items, horizon=1, in_feat=in_feat,
                         emb_dim=EMB_DIM, num_levels=NUM_LEVELS, hidden_ch=HIDDEN_CH,
                         kernel_size=KERNEL_SIZE, dropout=DROPOUT).to(DEVICE)

    # 여기만 변경: fit_model → fit_model_h1
    model_h1 = fit_model_h1(model_h1, dl_tr, dl_vl, epochs=EPOCHS, lr=LR, device=DEVICE)
    return model_h1


In [23]:

# =========================
# 메인 파이프라인
# =========================
set_seed(SEED)

# 1) Load & prep train
train_df = pd.read_csv(TRAIN_CSV)
train_df[TIME_COL] = pd.to_datetime(train_df[TIME_COL])
train_df = normalize_id_columns(train_df, ID_COL)
train_df = train_df.sort_values([ID_COL, TIME_COL])
train_df = add_calendar_features(train_df)


FEAT_COLS = ["dow_sin","dow_cos","mon_sin","mon_cos","is_weekend","holidays"]

# 2) ID mapping
uniq_ids = sorted(train_df[ID_COL].unique().tolist())
id2idx = {k: i + 1 for i, k in enumerate(uniq_ids)}
num_items = len(id2idx) + 1

# 3) Train models per mode
model_mimo = None
model_h1   = None
if PREDICT_MODE in ("mimo", "blend"):
    model_mimo = train_h7_model(train_df, FEAT_COLS, lookback=LOOKBACK, val_ratio=VAL_RATIO)
if PREDICT_MODE in ("recursive", "blend"):
    model_h1   = train_h1_model(train_df, FEAT_COLS, lookback=LOOKBACK, val_ratio=VAL_RATIO)

# 4) Inference (strict rules)
sample = pd.read_csv(SAMPLE_SUB)
raw_item_cols = sample.columns.tolist()[1:]
norm_to_raw = {_norm_id(c): c for c in raw_item_cols}

if model_mimo is not None: model_mimo.eval()
if model_h1  is not None: model_h1.eval()

test_pred_map = {}
with torch.no_grad():
  for path in sorted(glob.glob(TEST_GLOB)):
      base = os.path.basename(path).replace(".csv", "")
      df = pd.read_csv(path)
      df[TIME_COL] = pd.to_datetime(df[TIME_COL])
      df = normalize_id_columns(df, ID_COL)
      df = df.sort_values([ID_COL, TIME_COL])
      df = add_calendar_features(df)

      for k_norm, g in df.groupby(ID_COL, sort=False):
          raw_col = norm_to_raw.get(k_norm)
          if raw_col is None:
              continue

          g = g.sort_values(TIME_COL)
          if len(g) < LOOKBACK:
              continue

          g_last = g.iloc[-LOOKBACK:]
          num_mat  = g_last[[TARGET_COL]].values.astype(np.float32)
          feat_mat = g_last[FEAT_COLS].values.astype(np.float32)
          last_date = g_last[TIME_COL].iloc[-1]
          id_idx = id2idx.get(k_norm, 0)

          if PREDICT_MODE == "mimo":
              pred7 = np.array(
                  model_mimo(
                      torch.from_numpy(num_mat[None, ...]).to(DEVICE),
                      torch.from_numpy(feat_mat[None, ...]).to(DEVICE),
                      torch.from_numpy(np.array([id_idx], dtype=np.int64)).to(DEVICE)
                  ).detach().cpu().numpy()[0],
                  dtype=np.float32
              )
          elif PREDICT_MODE == "recursive":
              pred7 = recursive_predict_7days(model_h1, num_mat, feat_mat, id_idx, last_date, FEAT_COLS)
          else:  # blend
              p_mimo = np.array(
                  model_mimo(
                      torch.from_numpy(num_mat[None, ...]).to(DEVICE),
                      torch.from_numpy(feat_mat[None, ...]).to(DEVICE),
                      torch.from_numpy(np.array([id_idx], dtype=np.int64)).to(DEVICE)
                  ).detach().cpu().numpy()[0],
                  dtype=np.float32
              )
              p_rec  = recursive_predict_7days(model_h1, num_mat, feat_mat, id_idx, last_date, FEAT_COLS)
              pred7  = 0.5 * p_mimo + 0.5 * p_rec

          test_pred_map[(base, raw_col)] = pred7.astype(np.float32)

# 5) Fill submission
out_df = sample.copy()

def parse_row_tag(s: str):
    tag, plus = s.split("+")
    return tag, int(plus.replace("일", ""))

for i, row in out_df.iterrows():
    tag, day = parse_row_tag(row["영업일자"])
    for col in raw_item_cols:
        p = test_pred_map.get((tag, col))
        out_df.at[i, col] = float(p[day-1]) if (p is not None and 1 <= day <= HORIZON) else 0.0

out_df.to_csv(OUT_SUB, index=False, encoding="utf-8-sig")
print(f"Saved submission -> {OUT_SUB}")

H=7 Train windows: (86464, 28, 1) (86464, 28, 6) (86464, 7)
H=7 Valid windows: (9650, 28, 1) (9650, 28, 6) (9650, 7)


/tmp/ipython-input-720127970.py:36: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
/tmp/ipython-input-720127970.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == "cuda")):


[001] train=0.610603 | val=0.553437
[002] train=0.536435 | val=0.542819
[003] train=0.528070 | val=0.535084
[004] train=0.521523 | val=0.532438
[005] train=0.516537 | val=0.533344
[006] train=0.512054 | val=0.527020
[007] train=0.506937 | val=0.525625
[008] train=0.502518 | val=0.521897
[009] train=0.497527 | val=0.521934
[010] train=0.493695 | val=0.518068
[011] train=0.490606 | val=0.523460
[012] train=0.487500 | val=0.522869
[013] train=0.485110 | val=0.520908
[014] train=0.483329 | val=0.517907
[015] train=0.480480 | val=0.518629
[016] train=0.478007 | val=0.515586
[017] train=0.476361 | val=0.517442
[018] train=0.475228 | val=0.527135
[019] train=0.473032 | val=0.518954
[020] train=0.471371 | val=0.521653
[021] train=0.470256 | val=0.520117
[022] train=0.468541 | val=0.519273
[023] train=0.467074 | val=0.527910
[024] train=0.465460 | val=0.521471
[025] train=0.462823 | val=0.518149
[026] train=0.461572 | val=0.526208
[027] train=0.461455 | val=0.524064
Early stopped @ 27, best val

/tmp/ipython-input-3440814012.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
/tmp/ipython-input-3440814012.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[H1 001] train=0.759629 | val=0.549155
[H1 002] train=0.667285 | val=0.531564
[H1 003] train=0.657099 | val=0.528842
[H1 004] train=0.648881 | val=0.523436
[H1 005] train=0.643507 | val=0.524497
[H1 006] train=0.638151 | val=0.523295
[H1 007] train=0.634943 | val=0.519358
[H1 008] train=0.630128 | val=0.518941
[H1 009] train=0.626235 | val=0.515863
[H1 010] train=0.620578 | val=0.512900
[H1 011] train=0.615893 | val=0.516197
[H1 012] train=0.613820 | val=0.517649
[H1 013] train=0.609035 | val=0.517107
[H1 014] train=0.605537 | val=0.516342
[H1 015] train=0.600829 | val=0.519520
[H1 016] train=0.597188 | val=0.517108
[H1 017] train=0.594276 | val=0.518908
[H1 018] train=0.589733 | val=0.518733
[H1 019] train=0.586612 | val=0.514399
[H1 020] train=0.582690 | val=0.518258
[H1 021] train=0.579675 | val=0.523330
H1 Early stopped @ 21, best val=0.512900
Saved submission -> submission_tcn_blend_dayalpha_70-87-81-62-55-51-16.csv


/tmp/ipython-input-3608532620.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '11.131254196166992' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  out_df.at[i, col] = float(p[day-1]) if (p is not None and 1 <= day <= HORIZON) else 0.0
/tmp/ipython-input-3608532620.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.262020111083984' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  out_df.at[i, col] = float(p[day-1]) if (p is not None and 1 <= day <= HORIZON) else 0.0
/tmp/ipython-input-3608532620.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.502141952514648' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  

In [ ]:
# ===== Save checkpoints & meta =====
import os, json
SAVE_DIR = "."
os.makedirs(SAVE_DIR, exist_ok=True)

meta = {
    "id_keys": list(id2idx.keys()),     # 재현 가능한 매핑을 위해 키 목록 저장(정렬된 원본 순서)
    "LOOKBACK": LOOKBACK,
    "HORIZON": HORIZON,
    "FEAT_COLS": FEAT_COLS,
    "EMB_DIM": EMB_DIM,
    "NUM_LEVELS": NUM_LEVELS,
    "HIDDEN_CH": HIDDEN_CH,
    "KERNEL_SIZE": KERNEL_SIZE,
    "DROPOUT": DROPOUT
}
with open(os.path.join(SAVE_DIR, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

# 모델 가중치 저장 (있는 것만 저장)
if 'model_mimo' in globals() and model_mimo is not None:
    torch.save(model_mimo.state_dict(), os.path.join(SAVE_DIR, "tcn_h7.pt"))
if 'model_h1'   in globals() and model_h1   is not None:
    torch.save(model_h1.state_dict(),   os.path.join(SAVE_DIR, "tcn_h1.pt"))

print(f"Saved checkpoints to {SAVE_DIR}")


Saved checkpoints to /content/drive/MyDrive/LG_Aimers_MJ


# MIMO, Recursive, blend 성능 비교

In [25]:
# =========================================================
# 오프라인 롤링 백테스트로 MIMO/Recursive/Blend 비교
# =========================================================
@torch.no_grad()
def rolling_backtest(train_df, model_mimo=None, model_h1=None, anchors_per_item=6):
    if model_mimo is not None: model_mimo.eval()
    if model_h1  is not None: model_h1.eval()

    scores = {"mimo": [], "recursive": [], "blend": []}
    weights = {"mimo": [], "recursive": [], "blend": []}

    for item, g in train_df.groupby("영업장명_메뉴명", sort=False):
        g = g.sort_values("영업일자")
        vals = g["매출수량"].values.astype(np.float32)
        if len(vals) < LOOKBACK + HORIZON + 7:
            continue

        possible = list(range(LOOKBACK, len(vals) - HORIZON + 1, 7))
        anchor_idx = possible[-anchors_per_item:] if len(possible) >= anchors_per_item else possible
        if not anchor_idx:
            continue

        w_item = approx_shop_weight(item)

        for p in anchor_idx:
            last28 = g.iloc[p-LOOKBACK:p]
            y_true = g.iloc[p:p+HORIZON]["매출수량"].values.astype(np.float32)

            num_mat  = last28[["매출수량"]].values.astype(np.float32)
            feat_mat = last28[FEAT_COLS].values.astype(np.float32)
            last_dt  = last28["영업일자"].iloc[-1]
            ii = id2idx.get(item, 0)

            # MIMO 예측
            if model_mimo is not None:
                pm = model_mimo(
                    torch.from_numpy(num_mat[None, ...]).to(DEVICE),
                    torch.from_numpy(feat_mat[None, ...]).to(DEVICE),
                    torch.from_numpy(np.array([ii], dtype=np.int64)).to(DEVICE)
                ).detach().cpu().numpy()[0]
                s_m = smape_np(pm, y_true)
                if not np.isnan(s_m):
                    scores["mimo"].append(s_m); weights["mimo"].append(w_item)

            # 재귀 예측
            if model_h1 is not None:
                pr = recursive_predict_7days(model_h1, num_mat, feat_mat, ii, last_dt)
                s_r = smape_np(pr, y_true)
                if not np.isnan(s_r):
                    scores["recursive"].append(s_r); weights["recursive"].append(w_item)

            # 블렌드
            if (model_mimo is not None) and (model_h1 is not None):
                pb = 0.5 * pm + 0.5 * pr
                s_b = smape_np(pb, y_true)
                if not np.isnan(s_b):
                    scores["blend"].append(s_b); weights["blend"].append(w_item)

    def wavg(name):
        if not scores[name]:
            return None
        s = np.array(scores[name]); w = np.array(weights[name])
        return float(np.sum(s*w) / np.sum(w))

    print("=== Offline Rolling SMAPE (weighted by shop) ===")
    for k in ["mimo","recursive","blend"]:
        v = wavg(k)
        if v is not None:
            print(f"{k:9s}: {v:.6f}  (n={len(scores[k])})")
        else:
            print(f"{k:9s}: N/A")

# ------------------ 실행 ------------------
# 두 모델 다 학습되어 있으면 세 가지 모두 출력, 하나면 해당 것만 출력
rolling_backtest(train_df, model_mimo=model_mimo, model_h1=model_h1, anchors_per_item=6)


=== Offline Rolling SMAPE (weighted by shop) ===
mimo     : 0.507756  (n=1043)
recursive: 0.513916  (n=1043)
blend    : 0.505233  (n=1043)


# blend 비율 최적화

In [26]:
def find_best_alpha(pairs, step=0.05):
    # pairs: list of (y_true(7,), p_mimo(7,), p_rec(7,), weight)
    best_a, best_s = 0.5, float('inf')
    grid = np.arange(0.0, 1.0 + 1e-9, step)
    for a in grid:
        num = den = 0.0
        for y, pm, pr, w in pairs:
            s = smape_np(a*pm + (1-a)*pr, y)
            if not np.isnan(s): num += s*w; den += w
        if den > 0 and (num/den) < best_s:
            best_s, best_a = (num/den), a
    return best_a, best_s

def find_best_alpha_daywise(pairs, step=0.05, iters=3, shrink=True):
    grid = np.arange(0.0, 1.0 + 1e-9, step)

    # 1) 글로벌 스칼라 α로 초기화
    def _scalar(alpha):
        num = den = 0.0
        for y, pm, pr, w in pairs:
            s = smape_np(alpha*pm + (1.0-alpha)*pr, y)
            if not np.isnan(s): num += s*w; den += w
        return (num/den) if den > 0 else np.inf

    best_a = 0.5; best_s = _scalar(best_a)
    for a in grid:
        sc = _scalar(a)
        if sc < best_s: best_s, best_a = sc, a
    a = np.full(7, best_a, dtype=np.float32)

    # 2) 좌표강하로 요일별 α 벡터 탐색
    def _score(alpha_vec):
        num = den = 0.0
        for y, pm, pr, w in pairs:
            s = smape_np(alpha_vec*pm + (1.0-alpha_vec)*pr, y)  # element-wise
            if not np.isnan(s): num += s*w; den += w
        return (num/den) if den > 0 else np.inf

    best_s = _score(a)
    for _ in range(iters):
        improved = False
        for d in range(7):
            cur_best_v, cur_best_s = a[d], best_s
            for v in grid:
                a_try = a.copy(); a_try[d] = v
                sc = _score(a_try)
                if sc < cur_best_s:
                    cur_best_s, cur_best_v = sc, v
            if cur_best_v != a[d]:
                a[d] = cur_best_v; best_s = cur_best_s; improved = True
        if not improved: break

    # 3) 데이터가 적으면 글로벌 쪽으로 살짝 수축(선택)
    if shrink:
        n_pairs, lam = len(pairs), 20.0
        w = n_pairs / (n_pairs + lam)
        a = (w*a + (1.0 - w)*best_a).astype(np.float32)

    return a, float(best_s)

def smooth_alpha(a, times=1):
    a = a.astype(np.float32).copy()
    for _ in range(times):
        b = a.copy()
        for i in range(1, 6):  # 가운데만 보간
            b[i] = 0.25*a[i-1] + 0.5*a[i] + 0.25*a[i+1]
        a = b
    return a

In [27]:
# === α grid search (shop-weighted SMAPE 최소화) ===
if model_mimo is not None: model_mimo.eval()
if model_h1  is not None: model_h1.eval()

# 추가 : 과적합 방지
@torch.no_grad()
def collect_backtest_pairs(df_like, model_mimo, model_h1, anchors_per_item=6):
    pairs = []  # (y_true(7,), p_mimo(7,), p_rec(7,), weight)
    for item, g in df_like.groupby(ID_COL, sort=False):
        g = g.sort_values(TIME_COL)
        if len(g) < LOOKBACK + HORIZON + 7:
            continue
        possible = list(range(LOOKBACK, len(g) - HORIZON + 1, 7))
        idxs = possible[-anchors_per_item:] if len(possible) >= anchors_per_item else possible
        if not idxs:
            continue
        w = approx_shop_weight(item)

        for p in idxs:
            last28 = g.iloc[p-LOOKBACK:p]
            y = g.iloc[p:p+HORIZON][TARGET_COL].values.astype(np.float32)
            Xn = last28[[TARGET_COL]].values.astype(np.float32)
            Xf = last28[FEAT_COLS].values.astype(np.float32)
            ii = id2idx.get(item, 0)
            last_dt = last28[TIME_COL].iloc[-1]

            pm = model_mimo(
                torch.from_numpy(Xn[None,...]).to(DEVICE),
                torch.from_numpy(Xf[None,...]).to(DEVICE),
                torch.from_numpy(np.array([ii], dtype=np.int64)).to(DEVICE)
            ).detach().cpu().numpy()[0]

            pr = recursive_predict_7days(model_h1, Xn, Xf, ii, last_dt, FEAT_COLS)
            pairs.append((y, pm, pr, w))
    return pairs

# 추가 : 과적합 방지를 위해 α 계산용 보조 훈련
max_date = train_df[TIME_COL].max()
cutoff = max_date - pd.Timedelta(days=56)

train_cut  = train_df[train_df[TIME_COL] <= cutoff].copy()
calib_tail = train_df[train_df[TIME_COL] >  cutoff].copy()

if len(calib_tail) == 0 or calib_tail[TIME_COL].nunique() < 21:
    cutoff = max_date - pd.Timedelta(days=42)
    train_cut  = train_df[train_df[TIME_COL] <= cutoff].copy()
    calib_tail = train_df[train_df[TIME_COL] >  cutoff].copy()

print(f"[α-calib] cutoff={cutoff.date()} | train_cut rows={len(train_cut)} | calib_tail rows={len(calib_tail)}")

# 추가 : 과적합 방지
tmp_mimo = train_h7_model(train_cut, FEAT_COLS, lookback=LOOKBACK, val_ratio=VAL_RATIO)
tmp_h1   = train_h1_model(train_cut, FEAT_COLS, lookback=LOOKBACK, val_ratio=VAL_RATIO)

pairs = collect_backtest_pairs(calib_tail, tmp_mimo, tmp_h1, anchors_per_item=6)

def _score_alpha_vec(pairs, a_vec):
    num = den = 0.0
    for y, pm, pr, w in pairs:
        s = smape_np(a_vec*pm + (1.0 - a_vec)*pr, y)
        if not np.isnan(s): num += s*w; den += w
    return (num/den) if den>0 else float('inf')



[α-calib] cutoff=2024-04-20 | train_cut rows=91868 | calib_tail rows=10808
H=7 Train windows: (76621, 28, 1) (76621, 28, 6) (76621, 7)
H=7 Valid windows: (8685, 28, 1) (8685, 28, 6) (8685, 7)


/tmp/ipython-input-720127970.py:36: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
/tmp/ipython-input-720127970.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == "cuda")):


[001] train=0.607449 | val=0.623120
[002] train=0.531471 | val=0.615367
[003] train=0.522369 | val=0.603963
[004] train=0.516884 | val=0.592747
[005] train=0.512352 | val=0.590994
[006] train=0.508745 | val=0.592255
[007] train=0.504019 | val=0.585441
[008] train=0.500583 | val=0.577722
[009] train=0.496400 | val=0.564947
[010] train=0.493538 | val=0.563595
[011] train=0.490495 | val=0.558370
[012] train=0.487825 | val=0.550773
[013] train=0.485114 | val=0.553031
[014] train=0.482804 | val=0.552686
[015] train=0.479893 | val=0.542028
[016] train=0.478182 | val=0.547256
[017] train=0.476271 | val=0.542355
[018] train=0.474020 | val=0.543273
[019] train=0.472155 | val=0.539636
[020] train=0.471394 | val=0.538994
[021] train=0.468668 | val=0.537903
[022] train=0.467443 | val=0.532945
[023] train=0.465588 | val=0.532255
[024] train=0.464730 | val=0.528070
[025] train=0.463048 | val=0.538761
[026] train=0.461418 | val=0.528814
[027] train=0.459908 | val=0.531093
[028] train=0.457509 | val=0

/tmp/ipython-input-3440814012.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
/tmp/ipython-input-3440814012.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[H1 001] train=0.752345 | val=0.562453
[H1 002] train=0.664587 | val=0.566831
[H1 003] train=0.653817 | val=0.543915
[H1 004] train=0.647398 | val=0.546813
[H1 005] train=0.640305 | val=0.544740
[H1 006] train=0.637370 | val=0.550588
[H1 007] train=0.632554 | val=0.551623
[H1 008] train=0.627554 | val=0.540286
[H1 009] train=0.624243 | val=0.546075
[H1 010] train=0.618264 | val=0.542443
[H1 011] train=0.615865 | val=0.549617
[H1 012] train=0.609993 | val=0.550293
[H1 013] train=0.606718 | val=0.540959
[H1 014] train=0.605337 | val=0.541576
[H1 015] train=0.598819 | val=0.540040
[H1 016] train=0.595633 | val=0.545797
[H1 017] train=0.590358 | val=0.544842
[H1 018] train=0.587861 | val=0.546232
[H1 019] train=0.583350 | val=0.540148
[H1 020] train=0.578431 | val=0.543836
[H1 021] train=0.575422 | val=0.538245
[H1 022] train=0.573275 | val=0.539562
[H1 023] train=0.567073 | val=0.542877
[H1 024] train=0.564408 | val=0.542615
[H1 025] train=0.561409 | val=0.541327
[H1 026] train=0.558033 |

In [29]:
# 스칼라 α(체크용)
alpha_scalar, score_scalar = find_best_alpha(pairs, step=0.05)
a_rep = np.full(7, alpha_scalar, dtype=np.float32)
score_vec_when_scalar = _score_alpha_vec(pairs, a_rep)
print(f"[consistency] scalar={score_scalar:.6f} | vec(same-scalar)={score_vec_when_scalar:.6f}")

# 벡터 α(day-wise)
ALPHA_MIMO_VEC, score_pre = find_best_alpha_daywise(pairs, step=0.05, iters=3, shrink=True)
ALPHA_MIMO_VEC = smooth_alpha(ALPHA_MIMO_VEC, times=1).clip(0.0, 1.0)
score_post = _score_alpha_vec(pairs, ALPHA_MIMO_VEC)
print(f"[α-calib] Best α_vec={np.round(ALPHA_MIMO_VEC, 2)} | pre≈{score_pre:.6f} | post-smooth≈{score_post:.6f}")

del tmp_mimo, tmp_h1
torch.cuda.empty_cache()

[consistency] scalar=0.512831 | vec(same-scalar)=0.512831
[α-calib] Best α_vec=[0.5  0.28 0.43 0.4  0.27 0.36 0.35] | pre≈0.511905 | post-smooth≈0.512727


In [30]:
## 파이프라인이 제대로 연결되었는지 확인 & 재귀vs MIMO의 형태/값/간이 SMAPE 검증
# ==== Quick smoke test for recursive_predict_7days ====

# 전제: train_df에 캘린더 피처가 붙어 있어야 함
if not all(c in train_df.columns for c in FEAT_COLS):
    train_df = add_calendar_features(train_df)

# 예시 아이템 하나 잡기 (길이가 충분한 것)
item = None
for k, g in train_df.groupby(ID_COL, sort=False):
    if len(g) >= LOOKBACK + HORIZON + 7:
        item = k
        break
assert item is not None, "충분한 길이의 아이템이 없음"

g = train_df[train_df[ID_COL]==item].sort_values(TIME_COL)

# 앵커 포인트 선택 (마지막 예측 가능한 시점)
p = len(g) - HORIZON
assert p >= LOOKBACK, "LOOKBACK보다 짧음"

last28 = g.iloc[p-LOOKBACK:p]
y_true = g.iloc[p:p+HORIZON][TARGET_COL].values.astype(np.float32)

# 모델 입력 텐서 준비
Xn = last28[[TARGET_COL]].values.astype(np.float32)   # (28, 1)
Xf = last28[FEAT_COLS].values.astype(np.float32)      # (28, F)
ii = id2idx.get(item, 0)
last_dt = last28[TIME_COL].iloc[-1]

# 재귀 예측 호출 (5개 / 6개 인자 둘 다 OK)
pr5 = recursive_predict_7days(model_h1, Xn, Xf, ii, last_dt)
pr6 = recursive_predict_7days(model_h1, Xn, Xf, ii, last_dt, FEAT_COLS)
print("recursive (5 args):", pr5.shape, pr5[:3], " ...")
print("recursive (6 args):", pr6.shape, pr6[:3], " ...")
print("equal?", np.allclose(pr5, pr6))

# MIMO 모델도 있으면 같이 비교
if 'model_mimo' in globals() and model_mimo is not None:
    with torch.no_grad():
        p_mimo = model_mimo(
            torch.from_numpy(Xn[None,...]).to(DEVICE),
            torch.from_numpy(Xf[None,...]).to(DEVICE),
            torch.from_numpy(np.array([ii], dtype=np.int64)).to(DEVICE)
        ).detach().cpu().numpy()[0]
    print("mimo:", p_mimo.shape, p_mimo[:3], " ...")


print("SMAPE(recursive):", smape_np(pr5, y_true))
if 'model_mimo' in globals() and model_mimo is not None:
    print("SMAPE(mimo):", smape_np(p_mimo, y_true))

recursive (5 args): (7,) [7.079159 4.153673 3.935887]  ...
recursive (6 args): (7,) [7.079159 4.153673 3.935887]  ...
equal? True
mimo: (7,) [7.3300905 4.4627237 4.2301683]  ...
SMAPE(recursive): 0.9218041896820068
SMAPE(mimo): 0.9343698024749756


In [31]:
# === α grid-search로 최적 블렌드 가중치 찾기 & 제출 ===
# 최적 α로 테스트 세트 예측 → 제출
sample = pd.read_csv(SAMPLE_SUB)
out_df = sample.copy()
raw_item_cols = out_df.columns.tolist()[1:]
out_df[raw_item_cols] = out_df[raw_item_cols].astype('float32')
norm_to_raw = {_norm_id(c): c for c in raw_item_cols}

test_pred_map = {}
for path in sorted(glob.glob(TEST_GLOB)):
    base = os.path.basename(path).replace(".csv", "")
    df = pd.read_csv(path)
    df[TIME_COL] = pd.to_datetime(df[TIME_COL])
    df = normalize_id_columns(df, ID_COL)
    df = df.sort_values([ID_COL, TIME_COL])
    df = add_calendar_features(df)

    for k_norm, g in df.groupby(ID_COL, sort=False):
        raw_col = norm_to_raw.get(k_norm)
        if raw_col is None:
            continue

        g = g.sort_values(TIME_COL)
        if len(g) < LOOKBACK:
            continue

        g_last = g.iloc[-LOOKBACK:]
        Xn = g_last[[TARGET_COL]].values.astype(np.float32)
        Xf = g_last[FEAT_COLS].values.astype(np.float32)
        last_dt = g_last[TIME_COL].iloc[-1]
        ii = id2idx.get(k_norm, 0)

        with torch.no_grad():
            p_mimo = model_mimo(
                torch.from_numpy(Xn[None,...]).to(DEVICE),
                torch.from_numpy(Xf[None,...]).to(DEVICE),
                torch.from_numpy(np.array([ii],dtype=np.int64)).to(DEVICE)
            ).detach().cpu().numpy()[0].astype(np.float32)
        p_rec  = recursive_predict_7days(model_h1, Xn, Xf, ii, last_dt, FEAT_COLS)
        pred7 = (ALPHA_MIMO_VEC * p_mimo + (1.0 - ALPHA_MIMO_VEC) * p_rec).astype(np.float32)
        pred7 = np.nan_to_num(pred7, copy=False).clip(min=0.0)

        # --- 확신용 검증: day-wise α가 제대로 적용됐는지 ---
        _dbg = (ALPHA_MIMO_VEC * p_mimo + (1.0 - ALPHA_MIMO_VEC) * p_rec).astype(np.float32)
        _dbg = np.nan_to_num(_dbg, copy=False).clip(min=0.0)
        assert ALPHA_MIMO_VEC.shape == (7,), "ALPHA_MIMO_VEC must be shape (7,)"
        assert p_mimo.shape == (7,) and p_rec.shape == (7,), "Predictions must be shape (7,)"
        assert np.allclose(pred7, _dbg, atol=1e-6), "Day-wise alpha blend mismatch!"
        # ----------------------------------------------------

        test_pred_map[(base, raw_col)] = pred7

def parse_row_tag(s: str):
    tag, plus = s.split("+")
    return tag, int(plus.replace("일", ""))

for i, row in out_df.iterrows():
    tag, day = parse_row_tag(row["영업일자"])
    for col in raw_item_cols:
        p = test_pred_map.get((tag, col))
        out_df.at[i, col] = float(p[day-1]) if (p is not None and 1 <= day <= HORIZON) else 0.0

alpha_tag = "-".join([f"{int(round(x*100)):02d}" for x in ALPHA_MIMO_VEC])
OUT_SUB = f"submission_tcn_blend_dayalpha_{alpha_tag}.csv"
out_df.to_csv(OUT_SUB, index=False, encoding="utf-8-sig")
print(f"Saved submission -> {OUT_SUB}")

Saved submission -> submission_tcn_blend_dayalpha_50-28-43-40-27-36-35.csv
